In [42]:
import Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Printf

  Activating project at `~/devel/hw_ideas/mpc/hw1`


## Q3: Quadratic Programming Solver

In [43]:
function generate_qp(nx,ns,ny)
    nz = ns 
    x = randn(nx)
    Q = randn(nx,nx); Q = Q'*Q + I
    
    G = randn(ns,nx)
    s = abs.(randn(ns))
    z = abs.(randn(ns))
    for i = 1:ns 
        if rand() < 0.5 
            s[i] = 0 
        else
            z[i] = 0 
        end
    end
    h = G*x + s 
    A = randn(ny,nx)
    if rank(A) < size(A,1)
        A[1:ny,1:ny] += diagm(ones(ny))
    end
    b = A*x 
    y = randn(ny)
    q = -Q*x - G'*z - A'*y

    return Q,q,A,b,G,h,x,s,z,y
end

generate_qp (generic function with 1 method)

In [44]:
function unpackβ(β,idx)
    # unpack the β = [x;s;z;y]
    x = β[idx.x]
    s = β[idx.s]
    z = β[idx.z]
    y = β[idx.y]
    return x,s,z,y
end
function kkt_conditions(qp, idx, β, κ)
    x,s,z,y = unpackβ(β,idx)
    return [(qp.A'*y + qp.G'*z + qp.Q*x + qp.q); # stationarity
            (s .* z) .- κ;                       # relaxed complimentarity
            (qp.G*x + s - qp.h);                 # primal feasibility
            (qp.A*x - qp.b)]                     # primal feasibility
end

function linesearch(x,dx)
    # this returns the max α ∈ [0,1] st (x + Δx > 0)
    α = min(1.0, minimum([dx[i] < 0 ? -x[i]/dx[i] : Inf for i = 1:length(x)]))
    return α
end
let 

    nx = 10 
    ns = 5 
    nz = ns
    ny = 3 
    Q,q,A,b,G,h,x,s,z,y = generate_qp(10,5,3);
    x_true = 1*x
    s_true = 1*s
    z_true = 1*z 
    y_true = 1*y

    qp = (Q=Q, q=q, A=A, b=b, G=G, h=h)
    idx = (
        nx = nx,
        ns = ns, 
        nz = nz, 
        ny = ny, 
        x = 1:nx,
        s = (nx + 1) : (nx + ns),
        z = (nx + ns + 1) : (nx + ns + nz),
        y = (nx + ns + nz + 1) : (nx + ns + nz + ny)
        )

    # check KKT conditions
    println(norm(G*x + s - h))
    println(norm(A*x - b))
    println(minimum(s) >= 0 )
    println(minimum(z) >= 0 )
    println(norm(Q*x + q + G'*z + A'*y))
    println(norm(z .* s))


    # initialize variables 
    x = zeros(nx)
    y = zeros(ny)
    s = ones(ns)
    z = ones(nz)
    
    β = [x;s;z;y]

    # @show norm(kkt_conditions(qp, idx, β, 0.0))
    @printf "iter    μ         |Ax-b|     |Gx+s-h|    κ         α\n"
    @printf "--------------------------------------------------------\n"
    for pdip_iter = 1:10 

        # evaluate KKT conditions
        kkt_res = kkt_conditions(qp, idx, β, 0.0)

        # get jacobian of the KKT conditions
        kkt_jac = FD.jacobian(_β -> kkt_conditions(qp, idx, _β, 0.0), β)

        # solve for affine Newton step
        Δβ_a = -kkt_jac\kkt_res

        # unpack β
        x,s,z,y = unpackβ(β, idx)

        # unpack Δβ_a
        _, Δs_a, Δz_a, _ = unpackβ(Δβ_a, idx) 

        # get the current duality gap 
        μ = dot(s,z)

        # get the affine duality gap 
        α_a = min(linesearch(s, Δs_a), linesearch(z, Δz_a))
        μ_a = dot(s + α_a*Δs_a, z + α_a*Δz_a)

        # centering parameter 
        κ = μ*min(1, max(0, μ_a/μ))^3

        # get slackened kkt residual 
        kkt_res_relaxed = kkt_conditions(qp, idx, β, κ)

        # solve for affine Newton step
        Δβ_c = -kkt_jac\kkt_res_relaxed

        # unpack this second step direction 
        _, Δs_c, Δz_c, _ = unpackβ(Δβ_c, idx) 

        # linesearch 
        α = min(1, 0.99*min(linesearch(s, Δs_c), linesearch(z, Δz_c)))

        β += α*Δβ_c
        @printf("%3d  %9.2e  %9.2e  %9.2e  %9.2e  %6.4f\n",
          pdip_iter, sum(kkt_res[idx.s])/idx.ns, norm(kkt_res[idx.y]),
          norm(kkt_res[idx.z]), κ, α)


    end


end

0.0
0.0
true
true
2.381324868272655e-15
0.0
iter    μ         |Ax-b|     |Gx+s-h|    κ         α
--------------------------------------------------------
  1   1.00e+00   7.65e+00   6.80e+00   8.16e-02  0.7849
  2   2.77e-01   1.65e+00   1.46e+00   4.75e-02  0.9900
  3   9.15e-02   1.65e-02   1.46e-02   1.32e-03  0.9900
  4   1.47e-02   1.65e-04   1.46e-04   8.67e-04  0.9477
  5   3.02e-03   8.61e-06   7.65e-06   9.25e-05  0.9900
  6   5.04e-04   8.61e-08   7.65e-08   6.67e-06  0.9900
  7   7.07e-05   8.61e-10   7.65e-10   6.50e-08  0.9900
  8   4.36e-06   8.61e-12   7.65e-12   3.07e-12  0.9896
  9   6.57e-08   9.12e-14   7.87e-14   1.79e-19  0.9900
 10   6.62e-10   9.16e-16   2.48e-16   1.85e-27  0.9900


In [45]:
function unpackβ(β,idx)
    # unpack the β = [x;s;z;y]
    x = β[idx.x]
    z = β[idx.z]
    y = β[idx.y]
    return x,z,y
end
function c_eq(qp, x)
    qp.A*x - qp.b 
end
function h_ineq(qp, x)
    qp.G*x - qp.h
end
function kkt_conditions(qp, idx, β, κ)
    x,z,y = unpackβ(β,idx)
    return [
        qp.A'*y + qp.G'*z + qp.Q*x + qp.q; # stationarity
        c_eq(qp,x);
        (h_ineq(qp,x) .* z) .- κ
    ]
end


# function kkt_conditions(qp, idx, β, κ)
#     x,z,y = unpackβ(β,idx)
#     return [(qp.A'*y + qp.G'*z + qp.Q*x + qp.q); # stationarity
#             ((G*x - h) .* z) .- κ;                       # relaxed complimentarity
#             (qp.A*x - qp.b)]                     # primal feasibility
# end

function ineq_linesearch(qp, x, Δx)
    α = 1.0 
    for i = 1:10 
        x2 = x + α*Δx 
        if all(h_ineq(qp, x2) .< 0)
            return α
        else
            α /= 2
        end
    end
    error("ineq linesearch failed")
end
function positive_linesearch(x,dx)
    # this returns the max α ∈ [0,1] st (x + Δx > 0)
    α = min(1.0, minimum([dx[i] < 0 ? -x[i]/dx[i] : Inf for i = 1:length(x)]))
    return α
end
using Random  
# Random.seed!(MersenneTwister(1234), 1)
let 

    nx = 10 
    nz = 5 
    ny = 3 
    Q,q,A,b,G,h,x,s,z,y = generate_qp(nx,nz,ny);
#     jldsave("qp_data.jld2", Dict(""))
    x_true = 1*x
    z_true = 1*z 
    y_true = 1*y

    qp = (Q=Q, q=q, A=A, b=b, G=G, h=h)
    idx = (
        nx = nx,
        nz = nz, 
        ny = ny, 
        x = 1:nx,
        z = (nx + 1) : (nx + nz),
        y = (nx + nz + 1) : (nx + nz + ny)
        )


    x = G\h - G\ones(nz)
    z = ones(nz)
    y = zeros(ny)
    
    @assert all(h_ineq(qp, x) .< 0)
    @assert all(z .> 0)
        
    β = [x;z;y]
            
    @printf "iter    μ         |Ax-b|       κ         α\n"
    @printf "--------------------------------------------\n"
    for pdip_iter = 1:30 
        

        # evaluate KKT conditions
        kkt_res = kkt_conditions(qp, idx, β, 0.0)

        # get jacobian of the KKT conditions
        kkt_jac = FD.jacobian(_β -> kkt_conditions(qp, idx, _β, 0.0), β)

        # solve for affine Newton step
        Δβ_a = -kkt_jac\kkt_res

        # unpack β
        x,z,y = unpackβ(β, idx)
        @assert all(h_ineq(qp, x) .< 0)
        @assert all(z .> 0)

        # unpack Δβ_a
        Δx_a, Δz_a, Δy_a = unpackβ(Δβ_a, idx) 

        # get the current duality gap 
        μ = -dot(h_ineq(qp,x), z)
#         if μ < 1e-6 
#             @show μ
#             @info :success
#             break 
#         end

        # get the affine duality gap
        α_a = min(ineq_linesearch(qp, x, Δx_a), positive_linesearch(z, Δz_a))
        μ_a = -dot(h_ineq(qp, x + α_a*Δx_a), z + α_a*Δz_a)

        # centering parameter 
        κ = -μ*min(1, max(0, μ_a/μ))^3

        # get slackened kkt residual 
        kkt_res_relaxed = kkt_conditions(qp, idx, β, κ)

        # solve for affine Newton step
        Δβ_c = -kkt_jac\kkt_res_relaxed

        # unpack this second step direction 
        Δx_c, Δz_c, Δy_c = unpackβ(Δβ_c, idx) 

        # linesearch 
        α = min(1, 0.99*min(ineq_linesearch(qp, x, Δx_c), positive_linesearch(z, Δz_c)))

        β += α*Δβ_c
        @printf("%3d  %9.2e  %9.2e  %9.2e  %6.4f\n",
          pdip_iter, μ, norm(c_eq(qp,x)), κ, α)


    end


end

iter    μ         |Ax-b|       κ         α
--------------------------------------------
  1   5.00e+00   3.20e+00  -5.20e-01  0.4950
  2   3.42e+00   1.61e+00  -4.22e-01  0.9900
  3   1.72e+00   1.61e-02  -4.57e-03  0.9634
  4   2.54e-01   5.91e-04  -3.43e-02  0.9900
  5   1.77e-01   5.91e-06  -1.06e-05  0.9733
  6   8.58e-03   1.58e-07  -1.08e-03  0.9900
  7   5.44e-03   1.58e-09  -1.55e-11  0.9894
  8   6.20e-05   1.67e-11  -7.75e-06  0.9900
  9   3.90e-05   1.68e-13  -3.99e-20  0.9900
 10   3.90e-07   1.09e-15  -3.98e-28  0.9900
 11   3.90e-09   7.02e-16  -4.88e-10  0.9900
 12   2.45e-09   8.31e-16  -3.07e-10  0.9900
 13   1.54e-09   6.28e-16  -1.93e-10  0.9900
 14   9.70e-10   4.97e-16  -1.21e-10  0.9900
 15   6.10e-10   2.22e-16  -7.62e-11  0.9900
 16   3.83e-10   3.85e-16  -4.79e-11  0.9900
 17   2.41e-10   4.97e-16  -3.01e-11  0.9900
 18   1.52e-10   2.22e-16  -1.89e-11  0.9900
 19   9.53e-11   2.22e-16  -1.19e-11  0.9900
 20   5.99e-11   3.85e-16  -7.49e-12  0.9900
 21   3.77e-

In [114]:
function cost(qp::NamedTuple, x::Vector)::Real
    0.5*x'*qp.Q*x + dot(qp.q,x)
end
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end
function kkt_conditions(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector)::Vector
    return [
        qp.A'*λ + qp.G'*μ + qp.Q*x + qp.q; # stationarity
        c_eq(qp,x);
        (h_ineq(qp,x) .* μ)
    ]
end
function mask_matrix(qp::NamedTuple, x::Vector, μ::Vector, ρ::Real)::Matrix
    h = h_ineq(qp,x)
    Iρ = zeros(length(μ), length(μ))
    for i = 1:length(μ)
        if ((h[i] < 0) && (μ[i] ==0))
            Iρ[i,i]=0
        else
            Iρ[i,i]=ρ
        end
    end
    return Iρ
end
function augmented_lagrangian(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector, ρ::Real)::Real
    h = h_ineq(qp,x)
    c = c_eq(qp,x)
    Iρ = mask_matrix(qp, x, μ, ρ)
    cost(qp,x) + λ'*c + μ'*h + 0.5*ρ*c'c + 0.5*h'*Iρ*h
end
function logging(qp, main_iter, al_gradient, x, λ, μ, ρ)
    stationarity = norm(qp.A'*λ + qp.G'*μ + qp.Q*x + qp.q)
    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e\n",
          main_iter, norm(al_gradient), stationarity, maximum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(μ,h_ineq(qp,x))), ρ)
end
let 

    nx = 10 
    nμ = 5 
    nλ = 3 
    Q,q,A,b,G,h,x,_,μ,λ = generate_qp(nx,nμ,nλ);
    qp = (Q=Q, q=q, A=A, b=b, G=G, h=h)
    x_true = 1*x
    μ_true = 1*μ
    λ_true = 1*λ
    
    @show norm(kkt_conditions(qp, x, λ, μ))
    
    x = zeros(nx)
    λ = zeros(nλ)
    μ = zeros(nμ)
    ρ = 1.0 
    ϕ = 10.0
    
    @printf "iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ\n"
    @printf "----------------------------------------------------------------\n"
    for main_iter = 1:4
        g = FD.gradient(_x->augmented_lagrangian(qp,_x,λ,μ,ρ), x)
        if norm(g) < 1e-6 
            λ += ρ*c_eq(qp, x)
            μ = max.(0, μ + ρ*h_ineq(qp,x))
            ρ *= ϕ
            
        else
            H = FD.hessian(_x->augmented_lagrangian(qp,_x,λ,μ,ρ), x)
            x += -H\g
        end
        logging(qp, main_iter, 0, x, λ, μ, ρ)
    end
        
    
    
end
    
    

norm(kkt_conditions(qp, x, λ, μ)) = 3.688882027804963e-15
iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   0.00e+00   2.60e+00   6.11e-01   9.25e-01   0.00e+00  1e+00
  2   0.00e+00   2.60e+00   2.20e-01   9.31e-01   0.00e+00  1e+00
  3   0.00e+00   9.68e-15   2.20e-01   9.31e-01   4.86e-02  1e+01
  4   0.00e+00   3.07e+00   1.43e-02   8.94e-02   3.14e-03  1e+01


In [111]:
@printf "nothing in loop\n"
let 
    for i = 1:3
        g = FD.gradient(norm, ones(3))
        @printf "line printed in loop\n"
    end
end
# @printf "g = ones(3) in loop\n"
# let 
#     for i = 1:3
#         g = ones(3)
#         @printf "line printed in loop\n"
#     end
# end
# @printf "ForwardDiff call in loop\n"
# let 
#     for i = 1:3
#         g = FD.gradient(norm, ones(3))
#         @printf "line printed in loop\n"
#     end
# end

nothing in loop
line printed in loop
line printed in loop
line printed in loop
